## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Faanui,-16.4833,-151.7500,81.50,71,11,13.27,PF,few clouds
1,1,Kavieng,-2.5744,150.7967,82.29,76,90,7.78,PG,light rain
2,2,Severo-Kurilsk,50.6789,156.1250,35.40,92,82,3.38,RU,broken clouds
3,3,La Ronge,55.1001,-105.2842,23.00,46,55,4.61,CA,broken clouds
4,4,Thompson,55.7435,-97.8558,17.60,67,90,13.80,CA,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_minimum_temp = float(input("What is the minimum temperature you want?"))
user_maximum_temp = float(input("What is the maximum temperature you want?"))
#6.5.3

What is the minimum temperature you want?75
What is the maximum temperature you want?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
#6.5.3
filtered_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= user_maximum_temp) & \
                                    (city_data_df["Max Temp"] >= user_minimum_temp)]
filtered_city_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Faanui,-16.4833,-151.7500,81.50,71,11,13.27,PF,few clouds
1,1,Kavieng,-2.5744,150.7967,82.29,76,90,7.78,PG,light rain
5,5,Puerto Ayora,-0.7393,-90.3518,77.00,94,44,4.00,EC,scattered clouds
9,9,Rikitea,-23.1203,-134.9692,77.92,78,100,14.05,PF,light rain
14,14,Noumea,-22.2763,166.4572,81.00,73,90,10.36,NC,overcast clouds
15,15,Butaritari,3.0707,172.7902,82.98,72,70,8.81,KI,broken clouds
21,21,Atuona,-9.8000,-139.0333,80.08,81,87,20.89,PF,light rain
22,22,Vaini,-21.2000,-175.2000,86.00,79,75,4.61,TO,light rain
33,33,Bambous Virieux,-20.3428,57.7575,75.00,94,75,3.44,MU,broken clouds
37,37,Mayor Pablo Lagerenza,-19.9309,-60.7718,79.16,61,98,7.18,PY,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.*****
filtered_city_df.isnull().count()

#filtered_city_df.count()

City_ID        188
City           188
Lat            188
Lng            188
Max Temp       188
Humidity       188
Cloudiness     188
Wind Speed     188
Country        188
Description    188
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#6.5.3
clean_df = filtered_city_df.dropna()
clean_df.count()
#clean_df.columns

City_ID        188
City           188
Lat            188
Lng            188
Max Temp       188
Humidity       188
Cloudiness     188
Wind Speed     188
Country        188
Description    188
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Faanui,PF,81.50,few clouds,-16.4833,-151.7500,
1,Kavieng,PG,82.29,light rain,-2.5744,150.7967,
5,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,
9,Rikitea,PF,77.92,light rain,-23.1203,-134.9692,
14,Noumea,NC,81.00,overcast clouds,-22.2763,166.4572,
15,Butaritari,KI,82.98,broken clouds,3.0707,172.7902,
21,Atuona,PF,80.08,light rain,-9.8000,-139.0333,
22,Vaini,TO,86.00,light rain,-21.2000,-175.2000,
33,Bambous Virieux,MU,75.00,broken clouds,-20.3428,57.7575,
37,Mayor Pablo Lagerenza,PY,79.16,overcast clouds,-19.9309,-60.7718,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"] 
   
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"{row['City']} - Hotel not found... skipping")
        

Mayor Pablo Lagerenza - Hotel not found... skipping
Salto De Agua - Hotel not found... skipping
Barentu - Hotel not found... skipping
Araouane - Hotel not found... skipping
Kollo - Hotel not found... skipping
Gamba - Hotel not found... skipping
Nioro - Hotel not found... skipping
Kidal - Hotel not found... skipping
Saint-Louis - Hotel not found... skipping
Filingue - Hotel not found... skipping
Gorom-Gorom - Hotel not found... skipping


In [9]:
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Faanui,PF,81.50,few clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
1,Kavieng,PG,82.29,light rain,-2.5744,150.7967,Nusa Island Retreat
5,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Rikitea,PF,77.92,light rain,-23.1203,-134.9692,Pension Maro'i
14,Noumea,NC,81.00,overcast clouds,-22.2763,166.4572,Hôtel Le Lagon


In [10]:
# 7. Drop the rows where there is no Hotel Name
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City           188
Country        188
Max Temp       188
Description    188
Lat            188
Lng            188
Hotel Name     188
dtype: int64

In [11]:
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Faanui,PF,81.50,few clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
1,Kavieng,PG,82.29,light rain,-2.5744,150.7967,Nusa Island Retreat
5,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Rikitea,PF,77.92,light rain,-23.1203,-134.9692,Pension Maro'i
14,Noumea,NC,81.00,overcast clouds,-22.2763,166.4572,Hôtel Le Lagon


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
#fig = gmaps.figure()
#fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))